In [56]:
import cv2
import matplotlib.pyplot as plt
import os
import random
import shutil

In [57]:
def downscale_image(image,factor):
    old_dims = image.shape
    new_dims = [int(x * factor) for x in old_dims]
    small_image= cv2.resize(image, (new_dims[1], new_dims[0]))
    stretched_small_image = cv2.resize(small_image, (old_dims[1], old_dims[0]))
    return stretched_small_image

In [58]:
def show_images_side_by_side(lr, hr):
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(lr)
    plt.title('lr')

    plt.subplot(1, 2, 2)
    plt.imshow(hr)
    plt.title('hr')

    plt.show()

In [59]:
raw_images_folder = r'H:\my_files\my_programs\cat_upscaler\datasets\raw_cat_images'
image_paths = [os.path.join(raw_images_folder, f) for f in os.listdir(raw_images_folder) if os.path.isfile(os.path.join(raw_images_folder, f))]
random.shuffle(image_paths)
print(f'there are {len(image_paths)} images in the folder')

there are 5011 images in the folder


In [ ]:
def make_datum(image_path,export_image_dims):
    def calc_random_downscale_factors():
        def valid_factors(factors):
            #gap must not exceed 0.3
            #first num must be smaller
            #second num must be higher than 0.5

            highest_gap = 0.5
            min_value = 0.1

            if any([x < min_value for x in factors]):
                return False

            if abs(factors[0]-factors[1])>highest_gap:
                return False
            if factors[0] > factors[1]:
                return False

            return True

        while 1:
            factors = [random.randint(0,100) / 100 for _ in range(2)]
            if valid_factors(factors):
                return factors

    random_factors = calc_random_downscale_factors()
    image = cv2.imread(image_path)
    lr_image = downscale_image(image, random_factors[0])
    hr_image = downscale_image(image, random_factors[1])


    lr_image = cv2.resize(lr_image, (export_image_dims[1], export_image_dims[0]))
    hr_image = cv2.resize(hr_image, (export_image_dims[1], export_image_dims[0]))

    return lr_image,hr_image

def make_dataset(export_folder,export_image_dims ):
    hr_folder = os.path.join(export_folder, "hr")
    lr_folder = os.path.join(export_folder, "lr")

    if os.path.exists(export_folder):
        shutil.rmtree(export_folder)

    for folder in [export_folder, hr_folder, lr_folder]:
        if not os.path.exists(folder):
            os.makedirs(folder)

    for i,image_path in enumerate(image_paths):
        lr_image,hr_image=make_datum(image_path,export_image_dims)
        # show_images_side_by_side(lr_image, hr_image)
        fn = f'{i}.png'
        cv2.imwrite(os.path.join(hr_folder, fn), hr_image)
        cv2.imwrite(os.path.join(lr_folder, fn), lr_image)
        print(f'{i} / {len(image_paths)}')


make_dataset(r'H:\my_files\my_programs\cat_upscaler\datasets\cat_downscale_4',(2560,2560))

0 / 5011
1 / 5011
2 / 5011
3 / 5011
4 / 5011
5 / 5011
6 / 5011
7 / 5011
8 / 5011
9 / 5011
10 / 5011
11 / 5011
12 / 5011
13 / 5011
14 / 5011
15 / 5011
16 / 5011
17 / 5011
18 / 5011
19 / 5011
20 / 5011
21 / 5011
22 / 5011
23 / 5011
24 / 5011
25 / 5011
26 / 5011
27 / 5011
28 / 5011
29 / 5011
30 / 5011
31 / 5011
32 / 5011
33 / 5011
34 / 5011
35 / 5011
36 / 5011
37 / 5011
38 / 5011
39 / 5011
40 / 5011
41 / 5011
42 / 5011
43 / 5011
44 / 5011
45 / 5011
46 / 5011
47 / 5011
48 / 5011
49 / 5011
50 / 5011
51 / 5011
52 / 5011
53 / 5011
54 / 5011
55 / 5011
56 / 5011
57 / 5011
58 / 5011
59 / 5011
60 / 5011
61 / 5011
62 / 5011
63 / 5011
64 / 5011
65 / 5011
66 / 5011
67 / 5011
68 / 5011
69 / 5011
70 / 5011
71 / 5011
72 / 5011
73 / 5011
74 / 5011
75 / 5011
76 / 5011
77 / 5011
78 / 5011
79 / 5011
80 / 5011
81 / 5011
82 / 5011
83 / 5011
84 / 5011
85 / 5011
86 / 5011
87 / 5011
88 / 5011
89 / 5011
90 / 5011
91 / 5011
92 / 5011
93 / 5011
94 / 5011
95 / 5011
96 / 5011
97 / 5011
98 / 5011
99 / 5011
100 / 5011